In [5]:
# Copyright (c) OpenMMLab. All rights reserved.
import argparse
import os
from collections import Sequence
from pathlib import Path

import mmcv
import numpy as np
from mmcv import Config, DictAction
from mmcv.image import imwrite

from mmdet.core.utils import mask2ndarray
from mmdet.core.visualization import imshow_det_bboxes
from mmdet.datasets.builder import build_dataset
from mmdet.utils import replace_cfg_vals, update_data_root

import pandas as pd
import numpy as np
from tqdm import tqdm

In [6]:
def retrieve_data_cfg(config_path, skip_type, cfg_options):

    def skip_pipeline_steps(config):
        # skipping steps like normalization and defaultformat bundle, as they arent useful in creating augmented data
        config['pipeline'] = [
            x for x in config.pipeline if x['type'] not in skip_type
        ]

    cfg = Config.fromfile(config_path)

    # replace the ${key} with the value of cfg.key
    cfg = replace_cfg_vals(cfg)

    # update data root according to MMDET_DATASETS
    update_data_root(cfg)

    if cfg_options is not None:
        cfg.merge_from_dict(cfg_options)
    train_data_cfg = cfg.data.train
    while 'dataset' in train_data_cfg and train_data_cfg[
            'type'] != 'MultiImageMixDataset':
        train_data_cfg = train_data_cfg['dataset']

    if isinstance(train_data_cfg, Sequence):
        [skip_pipeline_steps(c) for c in train_data_cfg]
    else:
        skip_pipeline_steps(train_data_cfg)

    return cfg

In [7]:
def bulid_section_id(x):

    name = "{}@{}-{}-{}-{}"
    return name.format(x['file_name'], x['x_min'], x['x_max'],  x['y_min'], x['y_max'])  

In [8]:
#args = parse_args()
cfg = retrieve_data_cfg('augmentation_randomAffine+albu.py', ['DefaultFormatBundle', 'Normalize', 'Collect'] , None)

if 'gt_semantic_seg' in cfg.train_pipeline[-1]['keys']:
    cfg.data.train.pipeline = [
        p for p in cfg.data.train.pipeline if p['type'] != 'SegRescale'
   ]
    
print(cfg.data.train.pipeline)
dataset = build_dataset(cfg.data.train)

# progress_bar = mmcv.ProgressBar(len(dataset))
df_holder = []
count = 0

for item in tqdm(dataset):
    filename = os.path.join('../data/jpgs2/ra/',
                            'ra_'+Path(item['filename']).name
                            )
    
    
    gt_bboxes = item['gt_bboxes']
    gt_labels = item['gt_labels']
    gt_masks = item.get('gt_masks', None)
    
    if len(gt_bboxes) >0:
        
        # writing the augmented images
        imwrite(item['img'], filename)
        # taking the new augmented bounding boxes
        temp_df = pd.DataFrame(item['gt_bboxes'], columns = ['x_min', 'y_min', 'x_max', 'y_max'])
        temp_df['file_name'] = 'ra_'+Path(item['filename']).name
        height, width, channels = item['img'].shape
        temp_df['height'] = height
        temp_df['width'] = width
        temp_df['section_id'] = temp_df.apply(lambda x : bulid_section_id(x), axis=1)
        temp_df['staining'] = 'dummy'
        df_holder.append(temp_df)
        



[{'type': 'RandomAffine', 'scaling_ratio_range': (0.8, 2)}, {'type': 'Albu', 'transforms': [{'type': 'ShiftScaleRotate', 'shift_limit': 0.0625, 'scale_limit': 0.0, 'rotate_limit': 0, 'interpolation': 1, 'p': 0.5}, {'type': 'RandomBrightnessContrast', 'brightness_limit': [0.1, 0.3], 'contrast_limit': [0.1, 0.3], 'p': 0.2}, {'type': 'OneOf', 'transforms': [{'type': 'Blur', 'blur_limit': 3, 'p': 1.0}, {'type': 'MedianBlur', 'blur_limit': 3, 'p': 1.0}], 'p': 0.5}], 'bbox_params': {'type': 'BboxParams', 'format': 'pascal_voc', 'label_fields': ['gt_labels'], 'min_visibility': 0.0, 'filter_lost_elements': True}, 'keymap': {'img': 'image', 'gt_masks': 'masks', 'gt_bboxes': 'bboxes'}, 'update_pad_shape': False, 'skip_img_without_anno': False}, {'type': 'Resize', 'img_scale': (5120, 5120)}]
loading annotations into memory...
Done (t=0.43s)
creating index...
index created!


100%|██████████| 891/891 [36:24<00:00,  2.45s/it]


In [11]:
gt_df = pd.concat(df_holder, axis = 0)

gt_df.to_csv('../data/jpgs2/ra/ra_train.csv')

In [22]:
item

{'img_info': {'height': 8192,
  'width': 6024,
  'id': 36,
  'file_name': '117_D.jpg',
  'filename': '117_D.jpg'},
 'ann_info': {'bboxes': array([[2251., 4736., 2444., 4979.],
         [2296., 4499., 3059., 5016.],
         [2395., 4976., 2916., 5333.],
         [2574., 4254., 2802., 4502.],
         [2614., 2930., 3066., 3347.],
         [2683., 4394., 2976., 4619.],
         [2770., 3207., 3248., 3982.],
         [2829., 2429., 3670., 3119.],
         [2843., 2687., 3427., 3013.],
         [2879., 4306., 3050., 4520.],
         [2925., 4192., 3319., 4485.],
         [3177., 2972., 3864., 3468.],
         [3192., 3196., 3492., 3450.],
         [3217., 3355., 3764., 4036.],
         [3237., 4388., 3969., 5002.],
         [3685., 4568., 3997., 4760.],
         [3722., 3472., 4235., 3800.],
         [3833., 3190., 4192., 3500.],
         [4246., 3708., 4527., 4218.],
         [4255., 2971., 4454., 3251.],
         [4357., 3265., 4535., 3518.]], dtype=float32),
  'labels': array([0, 0, 0,

In [16]:
len(item['gt_bboxes'])

315